#### Notebook to feature engineer fraction of time PM10 was above EPA standard for all counties

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2018','2018_single_level_34_variables.nc')
sl_34_variables = xr.open_dataset(PATH_1) ## single level 34 variables
sl_34_variables
pm10=sl_34_variables['pm10'] 
pm10

<xarray.DataArray 'pm10' (valid_time: 2680, latitude: 34, longitude: 78)> Size: 28MB
[7107360 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 21kB 2018-01-31 ... 2018-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/33)
    GRIB_paramId:                             210074
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_units:                               kg m**-3
    long_name:                                Particulate matter d <= 10 um
    units:                                    kg m**-3
    standard_name:                            mass_concentration_of_pm10_ambi...
    GRIB_number:                              0
    GRIB_surface:                             0.0

In [3]:
pm10.to_dataframe()

pm10
valid_time          latitude longitude                              
2018-01-31 00:00:00 49.25    -124.849999999999994  0.000000010544120
                             -124.099999999999994  0.000000013048016
                             -123.349999999999994  0.000000014691853
                             -122.599999999999994  0.000000015076978
                             -121.849999999999994  0.000000014354516
...                                                              ...
2018-12-31 21:00:00 24.50    -70.099999999999994   0.000000002081954
                             -69.349999999999994   0.000000001700876
                             -68.599999999999994   0.000000001148440
                             -67.849999999999994   0.000000000871857
                             -67.099999999999994   0.000000000532168

[7107360 rows x 1 columns]

In [4]:
pm_threshold=150*10**(-9) ## epa standard
pm_threshold

1.5000000000000002e-07

In [5]:
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2018_county_shapefile','cb_2018_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


In [6]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-89.18137 37.04630, -89.17938 37.053...
1       POLYGON ((-84.44266 38.28324, -84.44114 38.283...
2       POLYGON ((-86.94486 37.07341, -86.94346 37.074...
3       POLYGON ((-84.12662 37.64540, -84.12483 37.646...
4       POLYGON ((-83.98428 38.44549, -83.98246 38.450...
                              ...                        
3228    POLYGON ((-100.09510 40.43866, -100.08937 40.4...
3229    POLYGON ((-82.22066 40.66758, -82.19327 40.667...
3230    POLYGON ((-99.30400 30.49983, -99.28234 30.499...
3231    POLYGON ((-88.06959 42.86726, -88.06959 42.872...
3232    POLYGON ((-86.26432 43.11830, -86.25103 43.118...
Name: geometry, Length: 3233, dtype: geometry

In [7]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0   -89.181368999999989  37.046304999999997
     1   -89.179383999999999  37.053011999999995
     2   -89.175725000000000  37.062069000000001
     3   -89.171880551089004  37.068183979028497
     4   -89.168087000000000  37.074217999999995
...                      ...                 ...
3232 148 -86.256650867846886  43.087448374873794
     149 -86.258404092491901  43.090980743149295
     150 -86.261183206056998  43.096580055604399
     151 -86.271995813165290  43.118365119164196
     152 -86.264320999999995  43.118296999999998

[1042614 rows x 2 columns]

#### Fraction of time PM10 was above EPA threshold

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time, PM2.5 was above EPA standard
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= pm10.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['pm10'].values[i] > pm_threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'PM10 above threshold':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3233

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 33min 52s
Wall time: 34min 20s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,PM10 above threshold
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3228,0.0
3229,0.0
3230,0.0
3231,0.0


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069
...,...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,PM10 above threshold
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007,0.0
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,0.0
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031,0.0
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065,0.0
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069,0.0
...,...,...,...,...
3228,Gosper,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073,0.0
3229,Holmes,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075,0.0
3230,Gillespie,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171,0.0
3231,Milwaukee,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079,0.0


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,PM10 above threshold
0,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007,0.0
1,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,0.0
2,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031,0.0
3,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065,0.0
4,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069,0.0


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,PM10 above threshold
0,Ballard,21007,0.000000000000000
1,Bourbon,21017,0.000000000000000
2,Butler,21031,0.000000000000000
3,Estill,21065,0.000000000000000
4,Fleming,21069,0.000000000000000
...,...,...,...
3228,Gosper,31073,0.000000000000000
3229,Holmes,39075,0.000000000000000
3230,Gillespie,48171,0.000000000000000
3231,Milwaukee,55079,0.000000000000000


In [16]:
county_var.to_pickle('PM10_above_threshold.pkl')